# Mode choice prediction for non-car owning households in the USA
**Decision-aid methodologies in transportation, EPFL Spring 2021**

Florent Zolliker, Gaelle Abi Younes, Luca Bataillard

## Step 1: Data pre-processing

In this step, we will process and adjust the dataset in order to facilitate our model training. We begin by importing the datasets and relevant libraries.

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SEED = 42
np.random.seed(SEED)

In [16]:
train_validate = pd.read_csv("nhts_train_validate_revised.csv", index_col="TRIPID")
test = pd.read_csv("nhts_test_revised.csv", index_col="TRIPID")

We now need to consider the features and their format in order to select the appropriate ones. 

We first notice a group of context columns, that are not relevant to model training:
* `TRIPID`: trip identifier, indexes the dataset but otherwise not relevant for training or grouped sampling.
* `HOUSEID`: household identifier, this is the topmost hiearchical group in the survey. This column should be used to perform grouped samping during cross-validation
* `PERSONID`: person identifier, this is another hiearchical group, but since `HOUSEID` is higher in the hiearchy, using it is not necessary.
* `TDTRPNUM`: trip numbering per person in the survey.

The target label is `TRAVELMODE`. This label is categorical, so we will encode the values using a simple numeric encoding. 

We also notice a `TRPTRANS` column that is very highly correllated with `TRAVELMODE` but does not feature in the `nhts_dictionary.csv` file provided with the project. After inspecting the NHTS documentation online, we suspect that the travel mode column was most likely generated from this column. We will thus discard the `TRPTRANS`column. Furthermore, negative responses in the `TRPTRANS` column resulted in `NaN` values in `TRAVELMODE`, that need to be filtered out.

Let us analyse the remaining columns. Missing values indicates that some values for that column are invalid in the dataset. In the case of categorical, this does not pose a big problem, since 

| Column name | Missing values | Categorical data | One-hot encoding | Scaling | Use feature? | Description | Comments |
| ---         | ---            | ---              | ---              | ---     | ---          | ---         | ---      |
| `STRTTIME`  |  -  |  -  |  -  | yes | yes | start time of trip | |
| `TRPMILES`  | yes |  -  |  -  | yes | yes | length of trip in miles ||
| `LOOP_TRIP` |  -  | yes | yes |  -  | yes | same origin and destination | binary variable, applying `mod 2` to this column can replace one-hot encoding |
| `TRIPPURP`  |  -  | yes | yes |  -  | yes | trip purpose ||
| `TRAVDAY`   |  -  | yes |  -  | yes | yes | weekday of travel ||
| `HOMEOWN`   | yes | yes | yes |  -  | yes | home ownership ||
| `HHSIZE`    |  -  |  -  |  -  | yes | yes | size of household ||
| `HHFAMINC`  | yes | yes |  -  | yes | yes | household income ||
| `HHSTATE`   |  -  | yes | yes |  -  |  ?  | household state of residency | could use either `HHSTATE` or `CENSUS_D` |
| `WRKCOUNT`  |  -  |  -  |  -  | yes | yes | number of workers in household ||
| `LIF_CYC`   |  -  |  -  |  -  | yes | yes | life cycle classification ||
| `URBAN`     |  -  | yes |  ?  |  ?  | yes | classification of urban area ||
| `URBANSIZE` |  -  | yes |  ?  |  ?  |  ?  | population size of urban area | redundant with `URBAN`, needs reordering or one-hot encoding |
| `CENSUS_D`  |  -  | yes | yes |  -  |  -  | census division (region) of household | could be redundant with `HHSTATE` |
| `HH_RACE`   | yes | yes | yes |  -  |  yes  | race of household | |
| `EDUC`      | yes | yes |  -  | yes |  yes  | educational attainment of household | |
| `WORKER`    | yes | yes | yes |  -  |  yes  | worker status | |
| `WHYTRP90`  | yes | yes | yes |  -  |  yes  | trip purpose with 1990 NPTS design | possible duplicate of `TRIPPURP`|
| `R_AGE_IMP` |  -  |  -  |  -  | yes |  yes  | age | |
| `R_SEX_IMP` |  -  | yes | yes |  -  |  yes  | gender | |
| `OBHUR`     | yes | yes |  -  |  -  |  yes  | urban/rural indicator at origin | |
| `DBHUR`     | yes | yes |  -  |  -  |  yes  | urban/rural indicator at destination | |
| `OBPPOPDN`  | yes | yes |  -  |  -  |   -   | population density at origin | already covered by `OBHUR` |
| `DBPPOPDN`  | yes | yes |  -  |  -  |   -   | population density at destination | already covered by `DBHUR` |














In [17]:
# Target column and topmost hiearchical sampling column
target = "TRAVELMODE"
group = "HOUSEID"

# Context columns (not used here) 
context_cols = [
    "TRIPID", "HOUSEID", "PERSONID", "TDTRPNUM"
]

# Columns used as features that do not need a specific encoding 
no_changes = [
    "STRTTIME", "TRPMILES", "TRAVDAY", "HHSIZE", "HHFAMINC", "WRKCOUNT", 
    "LIF_CYC", "URBAN", "EDUC", "R_AGE_IMP", "DBPPOPDN", "OBPPOPDN"
]

# Categorical columns that need one-hot encoding and the values they can take
one_hot_encodings = {
    "TRIPPURP": ["HBO", "HBSHOP", "HBSOCREC", "HBW", "NHB", -9],
    "HOMEOWN": [1, 2, 97, -8, -7],
    "HHSTATE": ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "IA", "ID", "IL", "IN", 
                   "KS","KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", "NJ","NM",
                    "NV","NY", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI", 
                    "WV", "WY"],
    "CENSUS_D": [c for c in range(1, 10)],
    "HH_RACE": [1, 2, 3, 4, 5, 6, 97, -7, -8], 
    "WORKER": [1, 2, -9, -1],
    "WHYTRP90": [1, 2, 3, 4, 5, 6, 8, 10, 11, 99],
    "R_SEX_IMP": [1, 2],
}

# Columns that need some in-place transformation (such as numerical label encoding) 
hur_encoding = lambda x: ["-9", "R", "T", "C", "S", "U"].index(x)
label_encodings = {
    "LOOP_TRIP": lambda x: x % 2,
    "URBANSIZE": lambda x: (x % 6) + 1,
    "OBHUR": hur_encoding,
    "DBHUR": hur_encoding,
}

In [18]:
def one_hot_encode(df, col_cat_encodings):
    """
    Takes a NHTS pandas dataset and a dictionary mapping categorical columns to their
    possible values. 
    Returns a new dataset with all categorical column one-hot encoding. A column of zeroes
    is created for each value that appears in the list of possible values but not in actual data
    
    TODO: Check out https://www.algosome.com/articles/dummy-variable-trap-regression.html
    """
    
    labels = set([col + ":" + str(cat) for col, cats in col_cat_encodings.items() for cat in cats])
    original_columns = set(df.columns)
    new_columns = (original_columns | labels) - col_cat_encodings.keys()
    
    df_one_hot = pd.get_dummies(df, columns=col_cat_encodings.keys(), prefix_sep=":")
    df_full = df_one_hot.reindex(columns=new_columns, fill_value=0)
    
    return df_full

def process_dataset(df):
    """
    Takes a pandas dataset in the NHTS survey format, keeps only columns in `no_changes`,
    `one_hot_encodings` and `label_encodings`. One-hot encodes features in `one_hot_encodings`
    and generates numeric labels for columns in `label_encodings`
    Returns a new dataframe containing the transformed features.
    """
    
    columns = [*no_changes, *one_hot_encodings.keys(), *label_encodings.keys()]
    features = df[columns]
    
    X = one_hot_encode(features, one_hot_encodings)
    for column, encoding in label_encodings.items():
        X[column] = X[column].map(encoding)
        
    return X

def process_target(y):
    return y.replace(target_encoding)

In [19]:
train_validate_no_nans = train_validate.dropna(axis="index")
test_no_nans = test.dropna(axis="index")

target_encoding = {mode: index for index, mode in 
                   enumerate(sorted(train_validate_no_nans[target].unique().tolist()))}

X = process_dataset(train_validate_no_nans)
y = process_target(train_validate_no_nans[target])
groups = train_validate_no_nans[group]

X_test = process_dataset(test_no_nans)

An additional step to our algorithm is to apply scaling to our values, which is needed with distance-based models such as logistic classifiers and k-nearest neighbours.

For indicator variables, no scaling is necessary. For other values, we can apply min-max or standard scaling. Standard scaling should be used when data is supposed to be normally distributed. Given that there are very few features where that assumption can be confidently made, we will apply min-max scaling to all scalable columns.

In [20]:
from sklearn.preprocessing import MinMaxScaler

X_scaled, X_test_scaled = X.copy(), X_test.copy()

scaled_columns = no_changes + list(label_encodings.keys())
scaler = MinMaxScaler()
scaler.fit(X_scaled[scaled_columns])



X_scaled[scaled_columns] = scaler.transform(X_scaled[scaled_columns])
X_test_scaled[scaled_columns] = scaler.transform(X_test_scaled[scaled_columns])

## Step 2: Hyperparameter search and model training

This project makes use of several heavy objects that take a significant amount of time to create. To reduce recomputation time when running the notebook again, we define a memoization utility function.

In [21]:
from functools import wraps
import os, joblib

def memoize(filename, func, *fargs, **fkwargs):
    """
    On first call, calls `func` with arguments `fargs` and `fkwargs`. Saves returned object to memo/`filename`.joblib.
    Subsequent calls retreive object from file. Returns saved object.
    """
    
    full_filename = "memo/" + filename + ".joblib"
    
    @wraps(func)
    def wrapper():
        if not os.path.isfile(full_filename):
            obj = func(*fargs, **fkwargs)
            joblib.dump(obj, full_filename)
            return obj
        else:
            return joblib.load(full_filename)
    
    return wrapper


We then define a generalised way of performing hyperparameter tuning. This will be applied to all models tried out throughout this notebook. We create a decorated memoized version of the same function.

In [22]:
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

def hyperparam_search(classifier, params, X=X, y=y, groups=groups, verbose=100, fit_params=None):
    """
    Takes a classifier, a dictionary of hyperparameter distributions and an (X,y,groups) dataset
    and returns a SearchCV object. The returned object is also saved to `filename` if it not none.
    Defaults to using non-scaled (X,y,groups).
    """
    
    gkf = GroupKFold(n_splits=5)
    search = RandomizedSearchCV(classifier, params,
                                scoring="neg_log_loss",
                                cv=gkf, 
                                n_jobs=-1, 
                                verbose=verbose, 
                                random_state=SEED)

    search.fit(X, y.squeeze(), groups=groups, **fit_params)
    
    return search

@wraps(hyperparam_search)
def m_hyperparam_search(filename, *fargs, **fkwargs):
    return memoize(filename, hyperparam_search, *fargs, **fkwargs)()

### Gradient Boosting

The first model we will consider is XGBoost. This model does not require feature scaling

In [ ]:
from xgboost import XGBClassifier
from scipy.stats import randint, uniform

xbg_params = {
    "n_estimators": randint(20, 500),
    "max_depth": randint(1, 5),
    "learning_rate": uniform(0.5,1),
    "reg_lambda": uniform(0.1, 2), 
}

xgb_fit_params = {
    'early_stopping_rounds': 10,
    'eval_set': [[X,y]]
}

xgb = XGBClassifier(objective="multi:softprob",
                    num_class=len(y.squeeze().unique()),
                    random_state=SEED,
                    verbose=0,
                    use_label_encoder=False,
                    eval_metric='mlogloss')


xgbsearch = m_hyperparam_search("xgb_search", xgb, xbg_params, fit_params=xgb_fit_params)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.6min


In [ ]:
xgbsearch.best_params_

In [ ]:
xgbsearch.best_score_

In [ ]:
best_xgb = xgbsearch.best_estimator_
best_xbg = memoize("best_xbg", lambda: xgbsearch.best_estimator_.fit(X, y))()

In [ ]:
y_pred = best_xbg.predict_proba(X_test)

In [ ]:
y_pred_pandas = pd.DataFrame(y_pred, columns=target_encoding.keys(), index=X_test.index)
y_pred_pandas.to_csv("best_xgb_pred.csv")

In [ ]:
y

In [ ]:
X_test.index

In [ ]:
X_test

In [ ]:
pd.DataFrame(xgbsearch.cv_results_)